In [1]:
import numpy as np
import pandas as pd
from sklearn import ensemble
from sklearn.linear_model import LinearRegression

from sklearn.cross_validation import cross_val_score
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn import manifold, decomposition, linear_model, ensemble, neighbors, cross_validation
import sklearn
from sklearn import pipeline, preprocessing, feature_extraction
#import xgboost as xgb
from sklearn.metrics import f1_score
#import xgboost as xgb
from sklearn.metrics import mean_absolute_error
import time
import datetime
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss, auc, roc_curve
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer
from scipy import sparse
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
#import lightgbm as lgbm
import scipy
from sklearn.preprocessing import StandardScaler
import cv2
from random import shuffle


%matplotlib inline
%pylab inline

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import gc
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

/usr/local/lib/python3.4/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python3.4/dist-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['shuffle']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [2]:
df_train = pd.read_csv('data/train_v2.csv')
labels = df_train['tags'].apply(lambda x: x.split(' '))

df_train.head()

,image_name,tags
0,train_0,haze primary
1,train_1,agriculture clear primary water
2,train_2,clear primary
3,train_3,clear primary
4,train_4,agriculture clear habitation primary road


In [3]:
sample_submission = pd.read_csv('data/sample_submission_v2.csv')


In [4]:
weather_labels = ['clear', 'partly_cloudy', 'haze', 'cloudy']
weather = []

for i in [l.split(' ') for l in df_train['tags'].values]:
    for j in i:
        if j in weather_labels:
            weather += [j]

df_train['weather'] = weather

In [5]:
skf = StratifiedKFold(4, shuffle=True, random_state=1)
folds = []
for itr, ite in skf.split(df_train, df_train.weather):
    folds += [[itr, ite]]

In [55]:
from keras.applications.vgg19 import VGG19
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Dropout
from keras import backend as K

# create the base pre-trained model
base_model = VGG19(weights='imagenet', include_top=False)

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
#x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(17, activation='sigmoid')(x)

# this is the model we will train
model = Model(input=base_model.input, output=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False


In [56]:
categories = np.array(['primary', 'artisinal_mine', 'cloudy','blooming','conventional_mine',
                       'partly_cloudy','road','water','selective_logging','blow_down',
                       'cultivation','agriculture','haze','habitation',
                       'slash_burn','bare_ground','clear'])

label_map = {l: i for i, l in enumerate(categories)}


#labels = list(set(flatten([l.split(' ') for l in df_train['tags'].values])))
#label_map = {l: i for i, l in enumerate(labels)}
weather_label_map = {l: i for i, l in enumerate(weather_labels)}
  


In [12]:
def all_generator(tr, batch_size):
    while(1):
        shuffle(tr)
        batches = []
        cnt = 0
        tmp_arr = []
        for i in tr:
            tmp_arr += [i]
            if cnt % batch_size == 0:
                batches += [tmp_arr]
                tmp_arr = []
            cnt += 1
        batches += [tmp_arr]
        print(len(batches))
        batch_num = 0
        for i in batches:
            batch_num += 1
            if batch_num % 50 == 0:
                print(batch_num)
            x_train = []
            y_train = []

            for f, tags, weather in df_train.ix[i, :].values:
                img = cv2.imread('data/train-jpg/{}.jpg'.format(f))
                targets = np.zeros(17)
                for t in tags.split(' '):
                    targets[label_map[t]] = 1
                #targets[weather_label_map[weather]] = 1
                img = cv2.resize(img, (224, 224))
                num_rows, num_cols = img.shape[:2]
                
                rotation_matrix = cv2.getRotationMatrix2D((num_cols/2, num_rows/2), random.choice([0, 90, 180, 270]), 1)
                img = cv2.warpAffine(img, rotation_matrix, (num_cols, num_rows))
                
                if random.choice([0, 1]) == 1:
                    img = cv2.flip( img, 0  )

                #1
                x_train.append(img)
                y_train.append(targets)  
            
            
            y_train = np.array(y_train, np.uint8)
            x_train = np.array(x_train, np.float16) / 255.0
            yield (x_train, y_train)

In [13]:
def val_generator(tr, batch_size, seed=0, shuf=1):
    while(1):
        if shuf==1:
            shuffle(tr)
        batches = []
        cnt = 0
        tmp_arr = []
        for i in tr:
            tmp_arr += [i]
            if cnt % batch_size == 0:
                batches += [tmp_arr]
                tmp_arr = []
            cnt += 1
        batches += [tmp_arr]
        
        for i in batches:
                
            x_train = []
            y_train = []

            for f, tags, weather in df_train.ix[i, :].values:
                img = cv2.imread('data/train-jpg/{}.jpg'.format(f))
                targets = np.zeros(17)
                for t in tags.split(' '):
                    targets[label_map[t]] = 1
                #targets[weather_label_map[weather]] = 1
                img = cv2.resize(img, (224, 224))
                num_rows, num_cols = img.shape[:2]
                
                rotation_matrix = cv2.getRotationMatrix2D((num_cols/2, num_rows/2), (seed % 4)*90, 1)
                img = cv2.warpAffine(img, rotation_matrix, (num_cols, num_rows))
                
                if (seed % 2 == 1):
                    img = cv2.flip( img, 0  )
                #1
                x_train.append(img)
                y_train.append(targets)  

            
            y_train = np.array(y_train, np.uint8)
            x_train = np.array(x_train, np.float16) / 255.0
            yield (x_train, y_train)

In [20]:
import keras
adam = keras.optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

In [57]:
from keras.callbacks import Callback, ModelCheckpoint

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(loss='binary_crossentropy', # We NEED binary here, since categorical_crossentropy l1 norms the output before calculating loss.
              optimizer=adam,
              metrics=['accuracy'])

model_checkpoint = ModelCheckpoint('VGG19.h5', save_best_only=True, save_weights_only=True)

In [58]:
batch_size = 128
model.fit_generator(all_generator(folds[3][0], batch_size),
                    samples_per_epoch=10000,
                    validation_data=val_generator(folds[3][1], batch_size),
                    nb_val_samples=5000,
                    nb_epoch=40,
                    verbose=2,
                    callbacks=[model_checkpoint])

Epoch 1/40239

50


/usr/local/lib/python3.4/dist-packages/Keras-1.2.2-py3.4.egg/keras/engine/training.py:1573: UserWarning: Epoch comprised more than `samples_per_epoch` samples, which might affect learning results. Set `samples_per_epoch` correctly to avoid this warning.
  warnings.warn('Epoch comprised more than '


256s - loss: 0.4745 - acc: 0.7925 - val_loss: 0.3145 - val_acc: 0.9052
Epoch 2/40
100
150
254s - loss: 0.3180 - acc: 0.8986 - val_loss: 0.2784 - val_acc: 0.9058
Epoch 3/40
200
239
254s - loss: 0.2873 - acc: 0.9047 - val_loss: 0.2682 - val_acc: 0.9049
Epoch 4/40
50
256s - loss: 0.2774 - acc: 0.9047 - val_loss: 0.2612 - val_acc: 0.9048
Epoch 5/40
100
150
254s - loss: 0.2697 - acc: 0.9057 - val_loss: 0.2582 - val_acc: 0.9060
Epoch 6/40
200
239
254s - loss: 0.2673 - acc: 0.9048 - val_loss: 0.2551 - val_acc: 0.9056
Epoch 7/40
50
253s - loss: 0.2647 - acc: 0.9049 - val_loss: 0.2529 - val_acc: 0.9052
Epoch 8/40
100
150
254s - loss: 0.2607 - acc: 0.9057 - val_loss: 0.2507 - val_acc: 0.9062
Epoch 9/40
200
239
254s - loss: 0.2567 - acc: 0.9064 - val_loss: 0.2485 - val_acc: 0.9064
Epoch 10/40
50
253s - loss: 0.2570 - acc: 0.9051 - val_loss: 0.2498 - val_acc: 0.9052
Epoch 11/40
100
150
255s - loss: 0.2573 - acc: 0.9052 - val_loss: 0.2471 - val_acc: 0.9057
Epoch 12/40
200
239
254s - loss: 0.2497 - 

KeyboardInterrupt: 

In [13]:
model.load_weights('resnet152_stage_2.h5')

In [60]:
#for layer in model.layers[:500]:
#    layer.trainable = False
for layer in model.layers[:]:
    layer.trainable = True

In [61]:
import keras
adam = keras.optimizers.Adam(lr=0.0001)


In [28]:
sgd = keras.optimizers.SGD(lr=0.000001, momentum=0.0, decay=0.0, nesterov=False)


In [62]:

model.compile(loss='binary_crossentropy', # We NEED binary here, since categorical_crossentropy l1 norms the output before calculating loss.
              optimizer=adam,
              metrics=['accuracy'])

# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
model_checkpoint = ModelCheckpoint('VGG19.h5', save_best_only=True, save_weights_only=True)




In [ ]:
batch_size = 64
model.fit_generator(all_generator(folds[3][0], batch_size),
                    samples_per_epoch=10000,
                    validation_data=val_generator(folds[3][1], batch_size),
                    nb_val_samples=5000,
                    nb_epoch=60,
                    verbose=2,
                    callbacks=[model_checkpoint])

Epoch 1/60
476
50
100
150


/usr/local/lib/python3.4/dist-packages/Keras-1.2.2-py3.4.egg/keras/engine/training.py:1573: UserWarning: Epoch comprised more than `samples_per_epoch` samples, which might affect learning results. Set `samples_per_epoch` correctly to avoid this warning.
  warnings.warn('Epoch comprised more than '


667s - loss: 0.1890 - acc: 0.9268 - val_loss: 0.1442 - val_acc: 0.9437
Epoch 2/60
200
250
300
666s - loss: 0.1464 - acc: 0.9441 - val_loss: 0.1311 - val_acc: 0.9504
Epoch 3/60
350
400
450
476
666s - loss: 0.1281 - acc: 0.9523 - val_loss: 0.1286 - val_acc: 0.9492
Epoch 4/60
50
100
150
668s - loss: 0.1354 - acc: 0.9494 - val_loss: 0.1225 - val_acc: 0.9546
Epoch 5/60
200
250
300
665s - loss: 0.1203 - acc: 0.9555 - val_loss: 0.1238 - val_acc: 0.9530
Epoch 6/60
350
400
450
476
666s - loss: 0.1168 - acc: 0.9564 - val_loss: 0.1131 - val_acc: 0.9581
Epoch 7/60
50
100
150
663s - loss: 0.1150 - acc: 0.9574 - val_loss: 0.1047 - val_acc: 0.9607
Epoch 8/60
200
250
300
665s - loss: 0.1089 - acc: 0.9596 - val_loss: 0.1097 - val_acc: 0.9583
Epoch 9/60
350
400
450
664s - loss: 0.1110 - acc: 0.9594 - val_loss: 0.1098 - val_acc: 0.9577
Epoch 10/60
476
50
100
150
663s - loss: 0.1068 - acc: 0.9607 - val_loss: 0.1035 - val_acc: 0.9608
Epoch 11/60
200
250
300
664s - loss: 0.1063 - acc: 0.9602 - val_loss: 0.1

In [ ]:
2

2

In [67]:
model.save_weights('fold4/VGG19.h5')

In [ ]:
1

1

In [ ]:
1

In [66]:
1

1

In [64]:
2

2

In [68]:
batch_size = 128
val_pred = model.predict_generator(val_generator(folds[3][1], batch_size, shuf=0, seed=0), val_samples = len(folds[3][1]) )

In [69]:
val_pred = val_pred * 0

In [70]:
batch_size = 128
for i in range(8):
    val_pred_tmp = model.predict_generator(val_generator(folds[3][1], batch_size, shuf=0, seed=i), val_samples = len(folds[3][1]) )
    val_pred = val_pred + val_pred_tmp
    print(i)

0
1
2
3
4
5
6
7


In [ ]:
val_pred

In [71]:
val_pred = val_pred / 8

In [72]:
val_pred = pd.DataFrame(val_pred)

In [73]:
val_pred['id'] = folds[3][1]
val_pred = val_pred.sort_values('id')
del val_pred['id']

In [74]:
val_pred.to_csv("fold4/train.csv", index=None)

In [47]:
v=pd.read_csv('fold4/train.csv')

In [10]:
len(val_pred)

13494

In [14]:
len(y_val)

13494

In [42]:
y_all = []
for f, tags, weather in df_train.values:
    targets = np.zeros(17)
    for t in tags.split(' '):
        targets[label_map[t]] = 1
    y_all += [targets]
y_all = pd.DataFrame(y_all)

In [39]:
y_all.shape

(40479, 17)

In [20]:
len(folds[0][1])

10122

In [43]:
y_val = y_all.ix[folds[0][1].tolist()]

/usr/local/lib/python3.4/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  """Entry point for launching an IPython kernel.


In [44]:
          
from sklearn.metrics import fbeta_score

#print(y_valid)
#print(p_valid)
print(fbeta_score(y_val, np.array(val_pred) > 0.2, beta=2, average='samples'))

0.923640800553


In [ ]:
0.922365582803
0.917041863094
0.913685914092
0.899545374775
0.89285374022
0.892722591666
0.889811292029
0.887170607068
0.863779970799

In [44]:
len(x_test)

61191

In [58]:
len(sample_submission)

61191

In [59]:
len(range(61191))

61191

In [75]:
def test_generator(tr, batch_size, seed=0):
    while(1):
        batches = []
        cnt = 0
        tmp_arr = []
        for i in tr:
            tmp_arr += [i]
            if cnt % batch_size == 0:
                batches += [tmp_arr]
                tmp_arr = []
            cnt += 1
        batches += [tmp_arr]
        
        for i in batches:
                
            x_test = []

            for f in sample_submission.ix[i, 'image_name'].values:
                img = cv2.imread('data/test-jpg/{}.jpg'.format(f))
                img = cv2.resize(img, (224, 224))
                num_rows, num_cols = img.shape[:2]
                
                rotation_matrix = cv2.getRotationMatrix2D((num_cols/2, num_rows/2), (seed % 4)*90, 1)
                img = cv2.warpAffine(img, rotation_matrix, (num_cols, num_rows))
                
                if (seed % 2 == 1):
                    img = cv2.flip( img, 0  )
                #1
                x_test.append(img)

            
            x_test = np.array(x_test, np.float16) / 255.0
            yield x_test

In [76]:
1

1

In [23]:
1

1

In [77]:
batch_size = 128
test_pred = model.predict_generator(test_generator(range(61191), batch_size, seed=0), val_samples = 61191)

In [78]:
result_pred = test_pred

In [ ]:
p_test = test_pred

In [54]:
import gc
gc.collect()

5924

In [79]:
batch_size = 128
for i in range(1,8):
    test_pred = model.predict_generator(test_generator(range(61191), batch_size, seed=i), val_samples = 61191 )
    result_pred = test_pred + result_pred
    print(i)

1
2
3
4
5
6
7


In [ ]:
2

In [56]:
1

1

In [82]:
result_pred = result_pred / 8

In [83]:
result_pred = pd.DataFrame(result_pred)

In [84]:
result_pred.to_csv("fold4/test.csv", index=None)

In [85]:
result_pred

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,0.999967,3.106106e-06,1.177439e-06,5.212919e-03,1.022755e-06,0.000925,0.000966,0.001429,5.062488e-04,2.432009e-04,0.001663,0.003297,0.000782,0.000320,3.407740e-05,0.000085,0.997537
1,0.999939,5.980464e-05,2.818454e-06,3.326960e-02,2.187197e-05,0.002569,0.004343,0.004395,7.766142e-03,3.785155e-03,0.012106,0.016742,0.000277,0.002726,6.262326e-04,0.000842,0.998239
2,0.998018,3.454607e-04,2.046244e-03,8.044354e-04,2.254558e-04,0.999746,0.010043,0.035536,5.562823e-04,9.467581e-04,0.017220,0.040024,0.000310,0.001847,4.917934e-04,0.001674,0.000099
3,0.999958,2.117519e-05,2.232488e-06,6.061943e-03,5.304804e-06,0.005796,0.003354,0.005371,1.944432e-03,1.700670e-03,0.134701,0.142489,0.000648,0.001855,1.017280e-03,0.001212,0.992230
4,0.361808,5.791064e-06,6.839007e-01,1.370201e-05,1.597645e-06,0.393776,0.002045,0.015132,7.307442e-06,9.923245e-06,0.001527,0.012225,0.003225,0.000380,1.202656e-05,0.000192,0.002534
5,0.999986,1.538210e-06,3.735647e-07,9.140233e-04,5.873340e-07,0.000666,0.000774,0.001212,1.840009e-04,1.301600e-04,0.001492,0.003057,0.000110,0.000227,2.120440e-05,0.000087,0.999115
6,0.999282,4.499687e-04,3.573338e-06,3.987618e-04,5.717833e-04,0.329551,0.246420,0.131417,1.550824e-03,3.542325e-04,0.426516,0.857581,0.033066,0.996704,1.001025e-02,0.008540,0.441267
7,0.678870,8.134326e-04,2.296823e-04,8.716126e-05,2.005903e-03,0.014306,0.964268,0.106803,5.682569e-04,8.303207e-05,0.018326,0.343445,0.015184,0.984827,9.150566e-04,0.034154,0.954906
8,0.999983,1.370272e-06,5.577581e-07,4.432974e-04,5.317090e-07,0.000692,0.000841,0.001376,1.112486e-04,9.040999e-05,0.001394,0.003040,0.000264,0.000196,1.716028e-05,0.000089,0.998565
9,0.994316,1.598539e-04,2.877838e-03,3.793644e-04,1.581127e-04,0.030063,0.044678,0.064213,6.814570e-04,1.514508e-03,0.162331,0.859225,0.585079,0.008042,2.259216e-03,0.011323,0.351896


In [80]:
1

1

In [45]:
p_test = np.array(p_test) > 0.2

In [46]:
p_test[:3]

array([[ True, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False,  True],
       [ True, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False,  True],
       [ True, False, False, False, False,  True, False, False, False,
        False, False, False, False, False, False, False, False]], dtype=bool)

In [47]:
label_map

{'agriculture': 11,
 'artisinal_mine': 1,
 'bare_ground': 15,
 'blooming': 3,
 'blow_down': 9,
 'clear': 16,
 'cloudy': 2,
 'conventional_mine': 4,
 'cultivation': 10,
 'habitation': 13,
 'haze': 12,
 'partly_cloudy': 5,
 'primary': 0,
 'road': 6,
 'selective_logging': 8,
 'slash_burn': 14,
 'water': 7}

In [48]:
categories = np.array(['primary', 'artisinal_mine', 'cloudy','blooming','conventional_mine',
                       'partly_cloudy','road','water','selective_logging','blow_down',
                       'cultivation','agriculture','haze','habitation',
                       'slash_burn','bare_ground','clear'])

In [49]:
res = []
for i in range(len(p_test)):
    res += [' '.join(categories[p_test[i]])]

In [33]:
len(res)

61191

In [50]:
sample_submission = pd.read_csv('data/sample_submission_v2.csv')

In [51]:
sample_submission['tags'] = res

In [52]:
sample_submission[:5]

,image_name,tags
0,test_0,primary clear
1,test_1,primary clear
2,test_2,primary partly_cloudy
3,test_3,primary cultivation agriculture clear
4,test_4,primary cloudy partly_cloudy


In [36]:
sample_submission[:5]

,image_name,tags
0,test_0,primary clear
1,test_1,primary clear
2,test_2,primary partly_cloudy
3,test_3,primary partly_cloudy cultivation agriculture ...
4,test_4,primary cloudy partly_cloudy


In [73]:
sample_submission[-5:]

,image_name,tags
61186,file_9995,primary partly_cloudy road water agriculture
61187,file_9996,primary road water cultivation agriculture hab...
61188,file_9997,primary clear
61189,file_9998,primary cloudy haze
61190,file_9999,primary partly_cloudy road water agriculture h...


In [53]:
sample_submission.to_csv("predictions/tmp13.csv", index=None)